<a href="https://colab.research.google.com/github/Otter2022/Oil_Marketing_Campaign/blob/main/tokens5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
#tokens5
import csv
import re
import sys
import os

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

source_data = '/content/drive/MyDrive/ColabNotebooks/FinalProject_AI/Source_Data/'
source_data_output = '/content/drive/MyDrive/ColabNotebooks/FinalProject_AI/Source_Data_Output/'
source_code = '/content/drive/MyDrive/ColabNotebooks/FinalProject_AI/Source_Code/'
admin = '/content/drive/MyDrive/ColabNotebooks/FinalProject_AI/Admin/'
backups = '/content/drive/MyDrive/ColabNotebooks/FinalProject_AI/Backups/'

#The lists contained in the marketing_compaign_oil.py file serves as a header for name pre-processing
#contains exclude, compiled_patterns for exclude, fix_abbreviations, honorifics, suffixes, suffixes_roman, etc.
marketing_campaign_oil = source_code+'marketing_campaign_oil.py'
with open(marketing_campaign_oil, 'r') as f:
    exec(f.read())
    f.close()

input_file = source_data + 'tokens5_2620_CSV.csv'
#input_file = basepath + 'tokens2_test_csv.csv'
output_file = source_data_output + 'tokens5_formatted_output.csv'

# Test for successul package import marketing_campaign_oil
print(honorifics)
print(suffixes)
print(suffixes_roman)

Mounted at /content/drive
['Dr', 'Doctor', 'Mr', 'Mrs', 'Ms', 'Miss', 'Prof', 'Professor', 'Sir', 'Madam', 'Dame', 'Lord', 'Lady', 'Rev', 'Reverend', 'Hon', 'Honorable', 'Capt', 'Captain', 'Major', 'Col', 'Colonel', 'Gen', 'General', 'Adm', 'Admiral', 'Sgt', 'Sergeant', 'Lt', 'Lieutenant', 'Chief', 'Officer', 'Judge', 'Magistrate', 'Sen', 'Senator', 'Rep', 'Representative', 'Amb', 'Ambassador', 'Pres', 'President', 'VP', 'Vice President', 'Gov', 'Governor', 'Mayor']
['Jr', 'Sr', 'Esq', 'Esquire', 'PhD', 'MD', 'DDS', 'DVM', 'JD', 'CPA', 'RN', 'LPN', 'PE', 'Ret']
['V', 'IV', 'III', 'II', 'I']


In [ ]:
def is_excluded(input_string):
    # Check if any compiled pattern matches the input string
    #I want to print the input_strings that are excluded
    for pattern in compiled_patterns:
        if pattern.match(input_string):
            print(f"Excluded: {input_string}")
            return True
    return False

def abbreviate(token):
  if token in fix_abbreviations.keys():
    return fix_abbreviations[token]
  else:
    return token


def rearrange_tokens(name):
    # Split the name into tokens
    tokens = name.split()

    # Process each token to fix abbreviations
    tokens = [abbreviate(token.strip()) for token in tokens]

    # Capitalize each token appropriately
    tokens = [token.capitalize() if token not in suffixes_roman else token for token in tokens]

    if len(tokens) == 5:
        token1, token2, token3, token4, token5 = tokens[0], tokens[1], tokens[2], tokens[3], tokens[4]

        # Exclude if any token for the owner name is in the exclude list
        for token in tokens:
          if is_excluded(token):
            print(f"Excluded: {name}")
            return "Miss"

        if token1 in honorifics and token2.isalpha() and token3.isalpha() and token4.isalpha() and (token5 in suffixes or token5 in suffixes_roman):#Mr. Smith John A (Jr/IV)
          return f"{token1} {token3} {token4} {token2} {token5}"
        elif token1 in honorifics and token2.isalpha() and token3.isalpha() and token4.isalpha() and (token5 not in suffixes and token5 not in suffixes_roman):#Mr. Smith John A (Jr/IV)
          return f"{token1} {token3} {token2} {token4} {token5}"
        elif re.search(r'[#\d\-]', token2):#token 2 contains at least 1 digit number, keep order
          return f"{token1} {token2} {token3} {token4} {token5}"
        elif token2 == '&':
          return f"{token3} {token4} {token2} {token1} {token5}"
        elif token2 == 'and':
          return f"{token1} {token2} {token3} {token4} {token5}"
        elif (token3 in suffixes or token3 in suffixes_roman) and (token4 == 'and' or token4 == '&'):
          return f"{token5} {token4} {token2} {token1} {token3}"
        elif token2 in fix_abbreviations.values() and token3 == "Trust":
          return f"{token1} {token2} {token3} {token4} {token5}"
        elif token2 == "Family":
          return f"{token1} {token2} {token3} {token4} {token5}"
        elif token2 != "Family" and token3 == "Trust":
          return f"{token2} {token1} {token3} {token4} {token5}"
        elif token2 not in fix_abbreviations.values() and re.search(r'[#\d\-]', token3):
          return f"{token2} {token1} {token3} {token4} {token5}"
        elif (token3 in fix_abbreviations.values()) and (token4 == "Trust"):
          return f"{token1} {token2} {token3} {token4} {token5}"
        elif (token3 not in fix_abbreviations.values()) and (token4 == "Trust"):
          return f"{token2} {token3} {token1} {token4} {token5}"
        elif (token3 == "&" or token3 == 'and') and  (token4 not in fix_abbreviations.values() and token5 not in fix_abbreviations.values()):
          return f"{token2} {token3} {token4} {token5} {token1}"
        elif re.search(r'[#\d\-]', token4):
          return f"{token2} {token3} {token1} {token4} {token5}"
        elif token4 == '&' or token4 == 'and':
          return f"{token2} {token3} {token4} {token5} {token1}"
        elif token5 == "Trust":
          return f"{token2} {token3} {token1} {token4} {token5}"
        else:
          return f"{token1} {token2} {token3} {token4} {token5}"#catch-all
    return "Miss"


def process_names(input_file, output_file):
    with open(input_file, 'r') as csvfile:
        reader = csv.reader(csvfile)
        # OwnerNames in 1st col
        names = [row[0] for row in reader]

    # Process the names
    names = [name.strip() for name in names]

    formatted_names = [(name, rearrange_tokens(name)) for name in names]
    misses = 0
    total = len(formatted_names)

    # Try different methods if the above one fails
    try:
        with open(output_file, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['column1', 'column2'])  # Write headers
            for original, rearranged in formatted_names:
                if rearranged == 'Miss':
                    misses += 1
                else:
                    writer.writerow([original, rearranged])
    except TypeError:
        with open(output_file, 'w') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['column1', 'column2'])  # Write headers
            for original, rearranged in formatted_names:
                if rearranged == 'Miss':
                    misses += 1
                else:
                    writer.writerow([original, rearranged])

    return misses, total



In [ ]:
def main():
    misses,total = process_names(input_file, output_file)
    print(f"Total: {total}")
    print(f"Errors: {misses}")
    print(f"Miss percentage {(float(misses)/float(total))*100.00}%.")

if __name__ == "__main__":
    main()

Excluded: Deceased
Excluded: GILL JOE P U/W/O DECEASED
Excluded: Corpus
Excluded: CORPUS CHRISTI MUS OF SCIENCE
Excluded: Deceased
Excluded: GREEN CHARLES C JR DECEASED
Excluded: Deceased
Excluded: HARWIT MARY BLOUNDT - DECEASED
Excluded: Deceased
Excluded: BRISTOW SARAH SOBOR - DECEASED
Excluded: Deceased
Excluded: PERRY W B JR DECEASED
Excluded: Energy-dorcus
Excluded: SM ENERGY-DORCUS JANEY (SA) W
Excluded: Energy-dorcus
Excluded: SM ENERGY-DORCUS WILMA (SA) J
Excluded: Management
Excluded: NEAL ROBERT L MANAGEMENT TRUST
Excluded: Fndt
Excluded: ST DAVID COMMUNITY HEALTH FNDT
Excluded: Survivors
Excluded: RAUCH ROBERT W SURVIVORS TRUST
Excluded: Survivors
Excluded: THW & ADW SURVIVORS TRUST
Excluded: Survivors
Excluded: WYCOFF ROBERT E SURVIVORS TR
Total: 2620
Errors: 13
Miss percentage 0.4961832061068702%.
